In [1]:
import os
from time import time

# 設定資料集檔案路徑
file_path = '/home/brian/Desktop/code/python code/IR/Pubmed 220 Train 2022-01-21.txt'

# 讀取資料集文件
with open(file_path, 'r', encoding='utf-8') as file:
    # 讀取並顯示前 5 篇文檔
    for i in range(5):
        print('----------------------------------------')
        print('Document', i+1)
        print(file.readline())  # 讀取一行內容

----------------------------------------
Document 1
###24491034

----------------------------------------
Document 2
BACKGROUND	The emergence of HIV as a chronic condition means that people living with HIV are required to take more responsibility for the self-management of their condition , including making physical , emotional and social adjustments .

----------------------------------------
Document 3
BACKGROUND	This paper describes the design and evaluation of Positive Outlook , an online program aiming to enhance the self-management skills of gay men living with HIV .

----------------------------------------
Document 4
METHODS	This study is designed as a randomised controlled trial in which men living with HIV in Australia will be assigned to either an intervention group or usual care control group .

----------------------------------------
Document 5
METHODS	The intervention group will participate in the online group program ` Positive Outlook ' .



In [6]:
import json

# 設定輸入文件路徑和輸出文件路徑
input_file_path = '/home/brian/Desktop/code/python code/IR/Pubmed 220 Train 2022-01-21.txt'
output_file_path = '/home/brian/Desktop/code/python code/IR/processed/pubmed_200k.jsonl'

# 打開輸入文件並處理每個文檔
with open(input_file_path, 'r', encoding='utf-8') as input_file, open(output_file_path, 'w', encoding='utf-8') as output_file:
    document_id = None
    document_content = ''
    for line in input_file:
        if line.startswith('###'):
            # 如果遇到新的文檔標識符，將上一個文檔寫入到輸出文件中
            if document_id is not None:
                json.dump({"id": document_id, "contents": document_content.strip()}, output_file)
                output_file.write('\n')  # 每個文檔後面換行
            # 更新文檔標識符和內容
            document_id = line.strip().replace('#', '')
            document_content = ''
        else:
            # 將行添加到文檔內容中
            document_content += line.strip() + ' '

    # 將最後一個文檔寫入到輸出文件中
    if document_id is not None:
        json.dump({"id": document_id, "contents": document_content.strip()}, output_file)


python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input "/home/brian/Desktop/code/python code/IR/processed/" \
  --index "/home/brian/Desktop/code/python code/IR/pubmed_200k_index" \
  --generator DefaultLuceneDocumentGenerator \
  --threads 1 \
  --storePositions --storeDocvectors --storeRaw

In [42]:
from pyserini.search.lucene import LuceneSearcher
import time

# 設置搜索器
searcher = LuceneSearcher('/home/brian/Desktop/code/python code/IR/pubmed_200k_index')

# 開始計時
start = time.time()

# 進行搜索
hits = searcher.search('cancer', k=100)

# 結束計時
end = time.time()

# 計算並輸出執行時間
print("執行時間：%f 秒" % (end - start))

for i in range(10):
    doc_id = hits[i].docid
    doc = searcher.doc(doc_id)
    print(f'Document {i+1}:')
    print(doc.raw())
    print('-------------------')

執行時間：0.005261 秒
Document 1:
{
  "id" : "24747090",
  "contents" : "OBJECTIVE\tWe examine the role of body mass index in the assessment of prostate cancer risk . METHODS\tA total of 3,258 participants who underwent biopsy ( including 1,902 men with a diagnosis of prostate cancer ) were identified from the Selenium and Vitamin E Cancer Prevention Trial . METHODS\tThe associations of body mass index with prostate cancer and high grade prostate cancer were examined using logistic regression , adjusting for age , race , body mass index adjusted prostate specific antigen , digital rectal examination , family history of prostate cancer , biopsy history , prostate specific antigen velocity , and time between study entry and the last biopsy . METHODS\tThe prediction models were compared with our previously developed body mass index adjusted Prostate Cancer Prevention Trial prostate cancer risk calculator . RESULTS\tOf the study subjects 49.1 % were overweight and 29.3 % were obese . RESULTS\tAf

In [41]:
import os
import json
import time

# 設定搜索關鍵字
search_key = 'cancer'

# 設定文件夾路徑
folder_path = '/home/brian/Desktop/code/python code/IR/processed' 

# 建立一個空列表用於存放搜索到的文檔
found_documents = []

# 開始計時
start = time.time()

# 設定計數器以限制找到的文檔數量
count = 0

# 遍歷文件夾中的每個文件
for filename in os.listdir(folder_path):
    if filename.endswith('.jsonl'):
        file_path = os.path.join(folder_path, filename)
        print('Processing file:', filename)  
        with open(file_path, 'r') as f:
            for line in f:
                json_data = json.loads(line.strip())
                # 檢查關鍵字是否在文檔內容中
                if search_key in json_data['contents']:
                    found_documents.append(json_data)
                    count += 1
                    # 如果已經找到10個文檔，則退出迴圈
                    if count == 100:
                        break

# 結束計時
end = time.time()

# 計算並輸出執行時間
print("執行時間：%f 秒" % (end - start))

Processing file: pubmed_200k.jsonl
執行時間：0.008557 秒


In [30]:
found_documents

[{'id': '16308853',
  'contents': 'BACKGROUND\tUse of blue dye alone as a marker for sentinel lymph node ( SLN ) biopsy is effective , but combining it with isotope marking can improve the success rate . BACKGROUND\tUse of the isotope adds extra cost and there are potential radiation hazards . BACKGROUND\tThe two techniques were compared in a randomized trial . METHODS\tWomen with early breast cancer ( less than 3 cm ) and no palpable axillary nodes were recruited . METHODS\tWomen older than 70 years with multicentric cancers or previous surgery to the breast or axilla were excluded . METHODS\tPatients were randomized to either blue dye alone or combined mapping for SLN biopsy . METHODS\tAll women had a level I and II axillary dissection after the SLN biopsy . RESULTS\tA total of 123 patients were recruited , of whom five were excluded from analysis . RESULTS\tBlue dye alone was used in 57 women and 61 had combined mapping . RESULTS\tBaseline demographic data were similar in the two co